In [1]:
import os
import os.path as osp
import numpy as np

import mmcv
from PIL import Image


root_path = '../../datasets/'

video_path = '/home/jinsuyoo/mmediting/jinsu/Focusing on the Economic Priorities for the Middle Class Nationwide-3vPdtajOJfw.mp4'
video = mmcv.VideoReader(video_path)
print(video.width, video.height, video.resolution, len(video))
video = video[1000:1100]

os.makedirs(osp.join(root_path, 'obama', 'hq_raw'), exist_ok=True)

for i, frame in enumerate(video):
    hq_raw = np.flip(frame, axis=2) / 255
    hq_raw = Image.fromarray((hq_raw * 255).astype(np.uint8))
    frame_idx = i + 1000
    hq_raw.save(osp.join(root_path, 'obama', 'hq_raw', f'{frame_idx:04}.png'))

/home/jinsuyoo/anaconda3/envs/vsr-lightning/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1280 720 (1280, 720) 6919


In [4]:
import glob
import dlib
import cv2
import os.path as osp
import sys
sys.path.append('/home/jinsuyoo/ResizeRight')
from resize_right import resize
import numpy as np

lq_size = 32

padding = 50


detector = dlib.get_frontal_face_detector() 
#sp = dlib.shape_predictor("shape_predictor_5_face_landmarks.dat") 
#facerec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")

img_path = sorted(glob.glob(osp.join(root_path, 'obama/hq_raw/*.png')))
save_path = osp.join(root_path, 'obama', 'lq', f'{lq_size}x{lq_size}')
os.makedirs(save_path, exist_ok=True)

avg_top, avg_bottom, avg_left, avg_right = [], [] ,[] ,[]

for ip in img_path:
    img = dlib.load_rgb_image(ip)
    dets = detector(img, 1)[0]
    avg_top.append(dets.top())
    avg_bottom.append(dets.bottom())
    avg_left.append(dets.left())
    avg_right.append(dets.right())
avg_top = int(np.mean(np.array(avg_top)))
avg_bottom = int(np.mean(np.array(avg_bottom)))
avg_left = int(np.mean(np.array(avg_left)))
avg_right = int(np.mean(np.array(avg_right)))

start_img_path = img_path[0]
start_img = dlib.load_rgb_image(start_img_path)
#start_img = np.array(Image.open(start_img_path).convert('RGB')) / 255

dets = detector(start_img, 1)[0]

for i, ip in enumerate(img_path):
    img = dlib.load_rgb_image(ip)
    #img = np.array(Image.open(ip).convert('RGB')) / 255

    crop = img[avg_top - padding:avg_bottom + padding, avg_left - padding:avg_right + padding]
    crop = crop / 255
    crop = np.clip(resize(crop, out_shape=(lq_size, lq_size)), 0, 1)
    crop = (crop * 255).round().astype(np.uint8)
    crop = crop[..., ::-1]

    frame_idx = i + 1000 
    cv2.imwrite(osp.join(save_path, f'{frame_idx:04}.png'), crop)